Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic regression

In [5]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Add L2 regularizer
    regularizer = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.363365
Minibatch accuracy: 6.2%
Validation accuracy: 8.0%
Minibatch loss at step 500: 0.844109
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 0.790811
Minibatch accuracy: 80.5%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 0.660986
Minibatch accuracy: 85.2%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 0.601978
Minibatch accuracy: 86.7%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 0.846742
Minibatch accuracy: 77.3%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.925859
Minibatch accuracy: 79.7%
Validation accuracy: 82.0%
Test accuracy: 88.6%


1 hidden layer neural network

In [7]:
batch_size = 128
beta = 0.000001

first_layer = image_size * image_size
second_layer = 1024
output_layer = 10
graph = tf.Graph()
with graph.as_default():
    
    # Input data: 
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights_1 = tf.Variable(tf.truncated_normal([first_layer, second_layer], stddev=0.1))
    biases_1 = tf.Variable(tf.zeros([second_layer]))
    weights_2 = tf.Variable(tf.truncated_normal([second_layer, output_layer], stddev=0.1))
    biases_2 = tf.Variable(tf.zeros([output_layer]))
    
    # Training computation
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    output_layer = tf.matmul(hidden_layer, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_layer))
    
    # Regularizer
    regularizer = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizor
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Prediction
    train_prediction = tf.nn.softmax(output_layer)
    valid_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, weights_2) + biases_2)
    test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer1, weights_2) + biases_2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate mini-batch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Create the dictionary
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if(step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.964071
Minibatch accuracy: 4.7%
Validation accuracy: 33.7%
Minibatch loss at step 500: 0.489376
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 1000: 0.488438
Minibatch accuracy: 85.2%
Validation accuracy: 86.8%
Minibatch loss at step 1500: 0.356354
Minibatch accuracy: 89.8%
Validation accuracy: 87.8%
Minibatch loss at step 2000: 0.322406
Minibatch accuracy: 91.4%
Validation accuracy: 87.9%
Minibatch loss at step 2500: 0.500268
Minibatch accuracy: 86.7%
Validation accuracy: 87.7%
Minibatch loss at step 3000: 0.421833
Minibatch accuracy: 88.3%
Validation accuracy: 88.7%
Test accuracy: 93.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 3001

train_dataset_short = train_dataset[:500, :]
train_labels_short = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        offset = (step * batch_size) % (train_labels_short.shape[0] - batch_size)
        # Generate mini-batch
        batch_data = train_dataset_short[offset:(offset + batch_size), :]
        batch_labels = train_labels_short[offset:(offset + batch_size), :]
        # Create the dictionary
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if(step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.084219
Minibatch accuracy: 8.6%
Validation accuracy: 26.6%
Minibatch loss at step 500: 0.005268
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 1000: 0.003908
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 0.003521
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 2000: 0.003389
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 0.003342
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 0.003304
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Test accuracy: 85.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
# Introduce dropout
batch_size = 128
beta = 0.0001

first_layer = image_size * image_size
second_layer = 1024
output_layer = 10
graph = tf.Graph()
with graph.as_default():
    
    # Input data: 
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights_1 = tf.Variable(tf.truncated_normal([first_layer, second_layer], stddev=0.1))
    biases_1 = tf.Variable(tf.zeros([second_layer]))
    weights_2 = tf.Variable(tf.truncated_normal([second_layer, output_layer], stddev=0.1))
    biases_2 = tf.Variable(tf.zeros([output_layer]))
    
    # Training computation
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    keep_prob = tf.placeholder(tf.float32)
    hidden_layer_dropout = tf.nn.dropout(hidden_layer, keep_prob)
    output_layer = tf.matmul(hidden_layer_dropout, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_layer))
    
    # Regularizer
    regularizer = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizor
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
    
    # Prediction
    train_prediction = tf.nn.softmax(output_layer)
    valid_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, weights_2) + biases_2)
    test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer1, weights_2) + biases_2)

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate mini-batch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Create the dictionary
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, keep_prob: 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if(step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.940503
Minibatch accuracy: 5.5%
Validation accuracy: 9.3%
Minibatch loss at step 500: 1.083149
Minibatch accuracy: 75.8%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 1.004371
Minibatch accuracy: 80.5%
Validation accuracy: 84.0%
Minibatch loss at step 1500: 0.875815
Minibatch accuracy: 86.7%
Validation accuracy: 84.8%
Minibatch loss at step 2000: 0.797970
Minibatch accuracy: 83.6%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 1.052972
Minibatch accuracy: 77.3%
Validation accuracy: 85.9%
Minibatch loss at step 3000: 1.004818
Minibatch accuracy: 84.4%
Validation accuracy: 86.3%
Test accuracy: 92.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Deep neural network function:

In [12]:
batch_size = 128
beta = 0.0001

first_layer_num = image_size * image_size
second_layer_num = 1024
third_layer_num = 500
fourth_layer_num = 200
output_layer_num = 10
graph = tf.Graph()
with graph.as_default():
    
    # Input data: 
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights_1 = tf.Variable(tf.truncated_normal([first_layer_num, second_layer_num], stddev=0.1))
    biases_1 = tf.Variable(tf.zeros([second_layer_num]))
    weights_2 = tf.Variable(tf.truncated_normal([second_layer_num, third_layer_num], stddev=0.1))
    biases_2 = tf.Variable(tf.zeros([third_layer_num]))
    weights_3 = tf.Variable(tf.truncated_normal([third_layer_num, fourth_layer_num], stddev=0.1))
    biases_3 = tf.Variable(tf.zeros([fourth_layer_num]))
    weights_4 = tf.Variable(tf.truncated_normal([fourth_layer_num, output_layer_num], stddev=0.1))
    biases_4 = tf.Variable(tf.zeros([output_layer_num]))
    
    # Training computation
    second_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    keep_prob = tf.placeholder(tf.float32)
    second_layer_dropout = tf.nn.dropout(second_layer, keep_prob)
    third_layer = tf.nn.relu(tf.matmul(second_layer_dropout, weights_2) + biases_2)
    third_layer_dropout = tf.nn.dropout(third_layer, keep_prob)
    fourth_layer = tf.nn.relu(tf.matmul(third_layer_dropout, weights_3) + biases_3)
    fourth_layer_dropout = tf.nn.dropout(fourth_layer, keep_prob)
    output_layer = tf.matmul(fourth_layer_dropout, weights_4) + biases_4
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_layer))
    
    # Regularizer
    regularizer = tf.nn.l2_loss(weights_1) \
                  + tf.nn.l2_loss(weights_2) \
                  + tf.nn.l2_loss(weights_3) \
                  + tf.nn.l2_loss(weights_4)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizor
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Prediction
    train_prediction = tf.nn.softmax(output_layer)
    
    valid_layer2 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_layer3 = tf.nn.relu(tf.matmul(valid_layer2, weights_2) + biases_2)
    valid_layer4 = tf.nn.relu(tf.matmul(valid_layer3, weights_3) + biases_3)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer4, weights_4) + biases_4)
    
    test_layer2 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_layer3 = tf.nn.relu(tf.matmul(test_layer2, weights_2) + biases_2)
    test_layer4 = tf.nn.relu(tf.matmul(test_layer3, weights_3) + biases_3)
    test_prediction = tf.nn.softmax(tf.matmul(test_layer4, weights_4) + biases_4)

In [13]:
num_steps = 30000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate mini-batch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Create the dictionary
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, keep_prob: 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if(step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 11.536633
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 500: 3.161776
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 3.110541
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 3.074188
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 3.039467
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 2500: 3.000912
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 3000: 2.962453
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 3500: 2.934756
Minibatch accuracy: 7.0%
Validation accuracy: 10.8%
Minibatch loss at step 4000: 2.817618
Minibatch accuracy: 16.4%
Validation accuracy: 18.7%
Minibatch loss at step 4500: 2.039562
Minibatch accuracy: 50.0%
Validation accuracy: 64.5%
Minibatch loss at step 5000: 1.514567
Minibatch accuracy: 71.1%
Validation accuracy: